In [2]:
from numpy import *
import numpy as np
import pandas as pd
import math as Math
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import random

In [3]:
import time
import datetime

def compute_time_interval(start, end):
   
    start = datetime.datetime.fromtimestamp(start / 1000.0)
    end = datetime.datetime.fromtimestamp(end / 1000.0)
    
    # 相减得到秒数
    seconds = (end- start).seconds
    
    return seconds

In [4]:
rc = 6378137
rj = 6356725
from math import atan, cos, asin, sqrt, pow, pi, sin
def rad(d):
    return d * math.pi / 180.0

def azimuth(pt_a, pt_b):
    lon_a, lat_a = pt_a
    lon_b, lat_b = pt_b
    rlon_a, rlat_a = rad(lon_a), rad(lat_a)
    rlon_b, rlat_b = rad(lon_b), rad(lat_b)
    ec=rj+(rc-rj)*(90.-lat_a)/90.
    ed=ec*cos(rlat_a)

    dx = (rlon_b - rlon_a) * ec
    dy = (rlat_b - rlat_a) * ed
    if dy == 0:
        angle = 90. 
    else:
        angle = atan(abs(dx / dy)) * 180.0 / pi
    dlon = lon_b - lon_a
    dlat = lat_b - lat_a
    if dlon > 0 and dlat <= 0:
        angle = (90. - angle) + 90
    elif dlon <= 0 and dlat < 0:
        angle = angle + 180 
    elif dlon < 0 and dlat >= 0:
        angle = (90. - angle) + 270 
    return angle

def distance(true_pt, pred_pt):
    lat1 = float(true_pt[1])
    lng1 = float(true_pt[0])
    lat2 = float(pred_pt[1])
    lng2 = float(pred_pt[0])
    radLat1 = rad(lat1)
    radLat2 = rad(lat2)
    a = radLat1 - radLat2
    b = rad(lng1) - rad(lng2)
    s = 2 * Math.asin(Math.sqrt(Math.pow(Math.sin(a/2),2) +
    Math.cos(radLat1)*Math.cos(radLat2)*Math.pow(Math.sin(b/2),2)))
    s = s * 6378.137
    s = round(s * 10000) / 10
    return s

def sq(x):
    return x*x

In [5]:
col_name_new = [
    #'Num_connected',
    'TrajID',
    'ECGI_1',
    'Dbm_1',
    'Freq_1',
    'CellID_1',
    'ECGI_2',
    'Dbm_2',
    'Freq_2',
    'CellID_2',
    'ECGI_3',
    'Dbm_3',
    'Freq_3',
    'CellID_3',
    'ECGI_4',
    'Dbm_4',
    'Freq_4',
    'CellID_4',
    'ECGI_5',
    'Dbm_5',
    'Freq_5',
    'CellID_5',
    'ECGI_6',
    'Dbm_6',
    'Freq_6',
    'CellID_6',
    'ECGI_7',
    'Dbm_7',
    'Freq_7',
    'CellID_7',
    #'RSSI_6',
]


In [6]:
col_name_rf = [
    'ECGI_1',
    'Dbm_1',
    'Freq_1',
    'CellID_1',
    'ECGI_2',
    'Dbm_2',
    'Freq_2',
    'CellID_2',
    'ECGI_3',
    'Dbm_3',
    'Freq_3',
    'CellID_3',
    'ECGI_4',
    'Dbm_4',
    'Freq_4',
    'CellID_4',
    'ECGI_5',
    'Dbm_5',
    'Freq_5',
    'CellID_5',
    'ECGI_6',
    'Dbm_6',
    'Freq_6',
    'CellID_6',
    'ECGI_7',
    'Dbm_7',
    'Freq_7',
    'CellID_7',
    'Lon','Lat','Lon2','Lat2','Lon3','Lat3','Lon4','Lat4','Lon5','Lat5','Lon6','Lat6','Lon7','Lat7'
]


In [14]:
def merge_2g_engpara():
    eng_para = pd.read_csv('huawei_big_data_gongcan_4g.csv', encoding='utf-8')
    eng_para = eng_para[['ECGI', 'PCI', '1F_UARFCN',u'经度',u'维度']]
    eng_para = eng_para.drop_duplicates()
    eng_para.rename(columns={u'经度': 'Lon', u'维度': 'Lat'}, inplace=True)
    eng_para['BSID'] = range(len(eng_para))
    eng_para['BSID'] = eng_para['BSID'].map(lambda x: x + 1)
    return eng_para

def make_rf_dataset(data, eng_para):
    for i in range(1, 8):
        data = data.merge(eng_para, left_on=['ECGI_%d' % i, 'Freq_%d' % i, 'CellID_%d' % i], right_on=['ECGI','1F_UARFCN','PCI'], 
                          how='left', suffixes=('', '%d' % i))
        temp=data['CellID_%d'% i].tolist()
        new=list()
        for item in temp:
            if math.isnan(item):
                new.append(0)
            elif int(item)<=0:
                new.append(0)
            else:
                new.append(item)
        data['CellID_%d' % i]=new
    data = data.fillna(-999.)
    #print data.columns
    
    feature = data[col_name_new+['MRTime','Timestamp','BSID','BSID2','BSID3','BSID4','BSID5','BSID6','BSID7','Longitude', 'Latitude',
                                 'Lon','Lat','Lon2','Lat2','Lon3','Lat3','Lon4','Lat4','Lon5','Lat5','Lon6','Lat6','Lon7','Lat7']]
   
    
    subset=[u'Longitude', u'Latitude', 
       u'ECGI_1', u'CellID_1',u'Freq_1',
       u'ECGI_2', u'CellID_2',u'Freq_2',
       u'ECGI_3', u'CellID_3',u'Freq_3',
       u'ECGI_4', u'CellID_4',u'Freq_4',
       u'ECGI_5', u'CellID_5',u'Freq_5',
       u'ECGI_6', u'CellID_6',u'Freq_6',
       u'ECGI_7', u'CellID_7',u'Freq_7',
       ]
    #feature=feature.drop_duplicates(subset=subset) 
    label = feature[['Longitude', 'Latitude']]
#     feature= feature.drop(['Longitude', 'Latitude'],axis=1)
    
    return feature, label

#eng_para = merge_2g_engpara()
eng_para =merge_2g_engpara()

In [8]:
def conf_model_label(error):
    conf_l=list()
   
    for t in error:
        if t<=50:
            conf_l.append(1)
        else:
            conf_l.append(0)
    
    return conf_l

In [9]:
data_2g=pd.read_csv("new_school.csv")

In [ ]:
# 1、邻接基站实验
bs_num = []
for idx,row in data_2g.iterrows():
    for i in range(2,8):
        if row['ECGI_%d'%i]==-999 or row['Freq_%d'%i]==-999 or row['CellID_%d'%i]==-999:
            bs_num.append(i-1)
            break
        if i == 7:
            bs_num.append(i)        
data_2g['bs_num'] = bs_num
retain_num = int(sys.argv[1])
for idx,row in data_2g.iterrows():
    if row['bs_num']-1 <= retain_num:
        continue
    for i in range(retain_num+2, row['bs_num']+1):
        data_2g.loc[idx, 'RNCID_%d'%i] = np.nan
        data_2g.loc[idx, 'CellID_%d'%i] = np.nan

In [ ]:
# 2、基站密度实验
bs = []
for i in range(1, 8):
    bs += data_2g[['ECGI_%d'% i, 'Freq_%d'% i, 'CellID_%d'% i]].values.tolist()
bs = [tuple(t) for t in bs]
temp = []
[temp.append(i) for i in bs if not i in temp]
bs = temp
ratio = float(sys.argv[1])
drop_bs = random.sample(bs, int(len(bs) * ratio))
for idx, row in data_2g.iterrows():
    for i in range(1, 7):
        if (row['ECGI_%d'% i], row['Freq_%d'% i], row['CellID_%d'% i]) in drop_bs:
            data_2g.loc[idx, 'ECGI_%d'% i] = -999
            data_2g.loc[idx, 'Freq_%d'% i] = -999
            data_2g.loc[idx, 'CellID_%d'% i] = -999
            data_2g.loc[idx, 'Dbm_%d'% i] = -999
data_2g = data_2g.drop(data_2g[data_2g['ECGI_1']==-999].index)            

In [16]:
train, label = make_rf_dataset(data_2g, eng_para)
from sklearn.cross_validation import train_test_split
tr_feature_r, te_feature_r, tr_label_, te_label_ = train_test_split(train, label, test_size=0.4,random_state=50)

In [17]:
def conf_pre(data):
    data=data.iloc[:,1:]
    label=data[['Longitude','Latitude']]
    data=data.drop(['Longitude', 'Latitude'],axis=1)
    
    return data, label

In [18]:
con_tr_feature, con_te_feature, con_tr_p, con_te_p = train_test_split(te_feature_r, te_label_, test_size=0.4)

In [19]:
tr_feature_r = tr_feature_r.sort_values(by='MRTime')
con_tr_feature = con_tr_feature.sort_values(by='MRTime')
con_te_feature = con_te_feature.sort_values(by='MRTime')
tr_label_ = tr_feature_r[['Longitude', 'Latitude']]
con_tr_p = con_tr_feature[['Longitude', 'Latitude']]
con_te_p = con_te_feature[['Longitude', 'Latitude']]

In [20]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor

In [21]:
#con_tr_feature.to_csv("2g/conf_tr_jd2g.csv")
#con_te_feature.to_csv("2g/conf_te_jd2g.csv")
#tr_feature_r.to_csv("2g/total_conf_tr_jd2g.csv")

In [22]:
import grid

rg = grid.RoadGrid(np.vstack((tr_label_.values, te_label_.values)),80)
tr_label_g = rg.transform(tr_label_.values, False) # label所在的grid
#rint tr_label_
con_tr_j = rg.transform(con_tr_p.values, False)
con_te_j = rg.transform(con_te_p.values, False)


## CCR

In [23]:
est=RandomForestClassifier( n_jobs=-1,
    n_estimators =50,
    max_features='sqrt'
).fit(tr_feature_r[col_name_rf].values, tr_label_g)

pred_tr=est.predict(tr_feature_r[col_name_rf].values)
tr_pred = np.array([rg.grid_center[idx] for idx in pred_tr])
error_tr = [distance(pt1, pt2) for pt1, pt2 in zip(tr_pred, tr_label_.values)]

pred_con_tr=est.predict(con_tr_feature[col_name_rf].values)
pred_con_te=est.predict(con_te_feature[col_name_rf].values)
tr_con_pred = np.array([rg.grid_center[idx] for idx in pred_con_tr])
te_con_pred = np.array([rg.grid_center[idx] for idx in pred_con_te])
error_con_tr = [distance(pt1, pt2) for pt1, pt2 in zip(tr_con_pred, con_tr_p.values)]
error_con_te = [distance(pt1, pt2) for pt1, pt2 in zip(te_con_pred, con_te_p.values)]

In [24]:
error_con_te = sorted(error_con_te)
np.median(error_con_te), np.mean(error_con_te), error_con_te[int(len(error_con_te) * 0.90)]

(46.899999999999999, 99.872253707376998, 242.9)

In [25]:
from scipy.sparse import csc_matrix

In [26]:
def feature_engineer(feature, pred, timestamp):
    add_feature = []
    timestamp_new=np.array(timestamp)
    for i in range(0, len(pred)):
        if i == 0:
            last_pt = pred[i]
            last_time = timestamp_new[i]
        else:
            last_pt = pred[i-1]
            last_time = timestamp_new[i-1]
        if i == len(pred)-1:
            next_pt = pred[i]
            next_time = timestamp_new[i]
        else:
            next_pt = pred[i+1]
            next_time = timestamp_new[i+1]
        sub_add_feature = []
        sub_add_feature.append(distance(last_pt, pred[i]))
        sub_add_feature.append(distance(next_pt, pred[i]))
        sub_add_feature.append(azimuth(last_pt, pred[i]))
        sub_add_feature.append(azimuth(pred[i], next_pt))
        sub_add_feature.append(timestamp_new[i]-last_time if timestamp_new[i]-last_time > 0 else 0)
        sub_add_feature.append(next_time-timestamp_new[i] if next_time-timestamp_new[i] > 0 else 0)
        sub_add_feature.append(sub_add_feature[0] / sub_add_feature[4] if sub_add_feature[4] > 0 else 0)
        sub_add_feature.append(sub_add_feature[1] / sub_add_feature[5] if sub_add_feature[5] > 0 else 0)
        sub_add_feature.append(last_pt[0])
        sub_add_feature.append(last_pt[1])
        sub_add_feature.append(next_pt[0])
        sub_add_feature.append(next_pt[1])
        sub_add_feature.append(pred[i][0])
        sub_add_feature.append(pred[i][1])
        #sub_add_feature.append(grid[i])
        add_feature.append(sub_add_feature)
    add_feature = np.asarray(add_feature)
    features = np.array(np.hstack((feature, add_feature)), dtype=float)
    feature = csc_matrix(features)
    return feature

In [27]:
feature_tr = feature_engineer(tr_feature_r[col_name_rf], tr_pred, tr_feature_r['Timestamp'])
feature_con_tr = feature_engineer(con_tr_feature[col_name_rf], tr_con_pred, con_tr_feature['Timestamp'])
feature_con_te = feature_engineer(con_te_feature[col_name_rf], te_con_pred, con_te_feature['Timestamp'])

est1=RandomForestClassifier( n_jobs=-1,
    n_estimators =50,
    max_features='sqrt'
).fit(feature_tr, tr_label_g)

pred_con_tr=est1.predict(feature_con_tr)
pred_con_te=est1.predict(feature_con_te)
tr_pred = np.array([rg.grid_center[idx] for idx in pred_con_tr])
te_pred = np.array([rg.grid_center[idx] for idx in pred_con_te])
error_tr = [distance(pt1, pt2) for pt1, pt2 in zip(tr_pred, con_tr_p.values)]
error_te = [distance(pt1, pt2) for pt1, pt2 in zip(te_pred, con_te_p.values)]

In [28]:
error_te = sorted(error_con_te)
np.median(error_te), np.mean(error_te), error_con_te[int(len(error_te) * 0.90)]

(46.899999999999999, 99.872253707376998, 242.9)

In [29]:
tr_feature_r['Longitude'] = tr_label_.iloc[:, 0].values
tr_feature_r['Latitude'] = tr_label_.iloc[:, 1].values
tr_feature_r['gid'] = tr_label_g

In [30]:
conf_label_tr = conf_model_label(error_tr)
conf_label_te = conf_model_label(error_te)

In [31]:
con_tr_feature["conf"]= conf_label_tr
con_te_feature["conf"]= conf_label_te
con_tr_feature["error"]= error_tr
con_te_feature["error"]= error_te

In [32]:
con_tr_feature['Longitude'] = con_tr_p.iloc[:,0]
con_tr_feature['Latitude'] = con_tr_p.iloc[:,1]
con_te_feature['Longitude'] = con_te_p.iloc[:,0]
con_te_feature['Latitude'] = con_te_p.iloc[:,1]

In [33]:
con_tr_feature['p_gid'] = pred_con_tr
con_tr_feature['gid'] = con_tr_j
con_te_feature['p_gid'] = pred_con_te
con_te_feature['gid'] = con_te_j

In [34]:
def rss_level(dbm):
    if dbm>-50:
        return 1
    elif dbm >-60:
        return 2
    elif dbm >-70:
        return 3
    elif dbm>-80:
        return 4
    elif dbm>-90:
        return 5
    elif dbm>-100:
        return 6
    elif dbm>-110:
        return 7
    else:
        return 8

In [35]:
for i in range(1, 8):
    con_tr_feature['rss_level_%d' % i] = con_tr_feature['Dbm_%d' % i].map(lambda x: rss_level(x))  
    con_te_feature['rss_level_%d' % i] = con_te_feature['Dbm_%d' % i].map(lambda x: rss_level(x))  

In [36]:
total_ob = con_tr_feature[['BSID','rss_level_1','BSID2','rss_level_2','BSID3','rss_level_3','BSID4','rss_level_4',
               'BSID5','rss_level_5','BSID6','rss_level_6','BSID7','rss_level_7']].drop_duplicates()

In [37]:
total_ob_bs = con_tr_feature[['BSID','BSID2','BSID3','BSID4','BSID5','BSID6','BSID7']].drop_duplicates()

In [38]:
total_ob_rss = con_tr_feature[['rss_level_1','rss_level_2','rss_level_3','rss_level_4','rss_level_5','rss_level_6',
                               'rss_level_7']].drop_duplicates()

In [39]:
total_ob_te = con_te_feature[['BSID','rss_level_1','BSID2','rss_level_2','BSID3','rss_level_3','BSID4','rss_level_4',
               'BSID5','rss_level_5','BSID6','rss_level_6','BSID7','rss_level_7']].drop_duplicates()


In [40]:
def jaccard_sim(list1, list2):
    #print list1, list2
    union_set = len(set(list1)|set(list2))#并集长度
    intersection_set = len(set(list1)&set(list2))#交集长度

    Jaccard = float(intersection_set/union_set) #Jaccar
    return Jaccard

In [41]:
def adaptive_emission_pro(jd_list, match, bs_list, ss_list, can_list, total_c, conf):
    pro_list = []
    weight_list = []
    if match.shape[0]>0:
        weight_list.append(math.log(1+match.shape[0])*1)
        match_ss= match[(match['rss_level_1']== int(ss_list[0])) & (match['rss_level_2']==int(ss_list[1])) & (match['rss_level_3']==ss_list[2])
             & (match['rss_level_4']==ss_list[3]) & (match['rss_level_5']==ss_list[4]) & (match['rss_level_6']==ss_list[5]) & (match['rss_level_7']==ss_list[6])]
        if match_ss.shape[0]>0:
            pro_list.append(float(match_ss[match_ss['conf']==conf].shape[0])/float(total_c))
        else:
            pro_list.append(float(match[match['conf']==conf].shape[0])/float(total_c))
        
        
    for can_temp, jd in zip(can_list, jd_list):
        match_c = con_tr_feature[(con_tr_feature['BSID']==int(can_temp[0])) & (con_tr_feature['BSID2']==int(can_temp[1]))
                      &(con_tr_feature['BSID3']==int(can_temp[2])) & (con_tr_feature['BSID4']==int(can_temp[3]))
                      &(con_tr_feature['BSID5']==int(can_temp[4])) & (con_tr_feature['BSID6']==int(can_temp[5]))
                                 & (con_tr_feature['BSID7']==int(can_temp[6]))]
        count = match_c.shape[0]
        weight_list.append(math.log(1+count)*jd)
        
        match_ss_c= match_c[(match_c['rss_level_1']== ss_list[0]) & (match_c['rss_level_2']== ss_list[1]) 
                            & (match_c['rss_level_3']==ss_list[2])& (match_c['rss_level_4']==ss_list[3]) 
                            & (match_c['rss_level_5']==ss_list[4]) & (match_c['rss_level_6']==ss_list[5])
                            & (match_c['rss_level_7']==ss_list[6])]
        if match_ss_c.shape[0]>0:
            pro_list.append(float(match_ss_c[match_ss_c['conf']==conf].shape[0])/float(total_c))
        else:
            pro_list.append(float(match_c[match_c['conf']==conf].shape[0])/float(total_c))


    weight_sum = np.sum(weight_list)
    ad_em_po = 0
    
    for x, y in zip(pro_list, weight_list):
        ad_em_po += x * (y / weight_sum)
    
    return ad_em_po

In [42]:
zeros_list = []
i =0
j=0
zero_num = con_tr_feature[con_tr_feature['conf']==0].shape[0]
for idx, row in total_ob_te.iterrows():
    bs_list =row[['BSID','BSID2','BSID3','BSID4','BSID5','BSID6','BSID7']].values
    ss_list = row[['rss_level_1','rss_level_2','rss_level_3','rss_level_4','rss_level_5','rss_level_6','rss_level_7']].values
    match = con_tr_feature[(con_tr_feature['BSID']==int(bs_list[0])) & (con_tr_feature['BSID2']==int(bs_list[1]))
                          &(con_tr_feature['BSID3']==int(bs_list[2])) & (con_tr_feature['BSID4']==int(bs_list[3]))
                          &(con_tr_feature['BSID5']==int(bs_list[4])) & (con_tr_feature['BSID6']==int(bs_list[5]))
                          &(con_tr_feature['BSID7']==int(bs_list[6]))]
    if match.shape[0]<5:
        can_bs_row = []
        can_j = []
        jaccd_max = []
        idx_list = []
        for idxx, roww in total_ob_bs.iterrows():
            can_bs_list = roww.values
            jd = jaccard_sim(bs_list, can_bs_list)
            jaccd_max.append(jd)
            idx_list.append(idxx)
            if jd>0.5:
                can_bs_row.append(can_bs_list)
                can_j.append(jd)
                
        if len(can_bs_row)==0:
            can_idx = jaccd_max.index(np.max(jaccd_max))
            can_temp = total_ob_bs.iloc[can_idx,:].values
            can_bs_row.append(total_ob_bs.iloc[can_idx,:].values)
            match_c = con_tr_feature[(con_tr_feature['BSID']==int(can_temp[0])) & (con_tr_feature['BSID2']==int(can_temp[1]))
                          &(con_tr_feature['BSID3']==int(can_temp[2])) & (con_tr_feature['BSID4']==int(can_temp[3]))
                          &(con_tr_feature['BSID5']==int(can_temp[4])) & (con_tr_feature['BSID6']==int(can_temp[5]))
                          &(con_tr_feature['BSID7']==int(can_temp[6]))]
            
            match_ss_c= match_c[(match_c['rss_level_1']== ss_list[0]) & (match_c['rss_level_2']== ss_list[1]) 
                            & (match_c['rss_level_3']==ss_list[2])& (match_c['rss_level_4']==ss_list[3]) 
                            & (match_c['rss_level_5']==ss_list[4]) & (match_c['rss_level_6']==ss_list[5])
                            & (match_c['rss_level_7']==ss_list[6])]
            if match_ss_c.shape[0]>0:
                zeros_list.append(float(match_ss_c[match_ss_c['conf']==0].shape[0])/float(zero_num))
            else:
                zeros_list.append(float(match_c[match_c['conf']==0].shape[0])/float(zero_num))
        
            j+=1 
        else:
            zeros_list.append(adaptive_emission_pro(can_j, match, bs_list, ss_list, can_bs_row, zero_num, 0))
        #print i, len(can_bs_row)
        i+=1
    else:
        j+=1
        match_ss= match[(match['rss_level_1']== int(ss_list[0])) & (match['rss_level_2']==int(ss_list[1])) & (match['rss_level_3']==ss_list[2])
             & (match['rss_level_4']==ss_list[3]) & (match['rss_level_5']==ss_list[4]) & (match['rss_level_6']==ss_list[5]) & (match['rss_level_7']==ss_list[6])]
        if match_ss.shape[0]>0:
            zeros_list.append(float(match_ss[match_ss['conf']==0].shape[0])/float(zero_num))
        else:
            zeros_list.append(float(match[match['conf']==0].shape[0])/float(zero_num))

In [39]:
zeros_list2 = np.array(zeros_list)
np.savetxt('zeros_list.txt', zeros_list2)
# zeros_list2 = np.loadtxt('zeros_list.txt')
# zeros_list2 = zeros_list2.tolist()

In [40]:
total_ob_te['conf_ad_em_pro_0'] = zeros_list2

In [41]:
one_list = []
one_num = con_tr_feature[con_tr_feature['conf']==1].shape[0]
for idx, row in total_ob_te.iterrows():
    bs_list =row[['BSID','BSID2','BSID3','BSID4','BSID5','BSID6','BSID7']].values
    ss_list = row[['rss_level_1','rss_level_2','rss_level_3','rss_level_4','rss_level_5','rss_level_6','rss_level_7']].values
    #print bs_list[0]
    match = con_tr_feature[(con_tr_feature['BSID']==int(bs_list[0])) & (con_tr_feature['BSID2']==int(bs_list[1]))
                          &(con_tr_feature['BSID3']==int(bs_list[2])) & (con_tr_feature['BSID4']==int(bs_list[3]))
                          &(con_tr_feature['BSID5']==int(bs_list[4])) & (con_tr_feature['BSID6']==int(bs_list[5]))
                          &(con_tr_feature['BSID7']==int(bs_list[6]))]
    if match.shape[0]<5:
        can_bs_row = []
        can_j = []
        jaccd_max = []
        idx_list = []
        for idxx, roww in total_ob_bs.iterrows():
            can_bs_list = roww.values
            jd = jaccard_sim(bs_list, can_bs_list)
            jaccd_max.append(jd)
            idx_list.append(idxx)
            if jd>0.5:
                can_bs_row.append(can_bs_list)
                can_j.append(jd)
                
        if len(can_bs_row)==0:
            can_idx = jaccd_max.index(np.max(jaccd_max))
            can_temp = total_ob_bs.iloc[can_idx,:].values
            can_bs_row.append(total_ob_bs.iloc[can_idx,:].values)
            match_c = con_tr_feature[(con_tr_feature['BSID']==int(can_temp[0])) & (con_tr_feature['BSID2']==int(can_temp[1]))
                          &(con_tr_feature['BSID3']==int(can_temp[2])) & (con_tr_feature['BSID4']==int(can_temp[3]))
                          &(con_tr_feature['BSID5']==int(can_temp[4])) & (con_tr_feature['BSID6']==int(can_temp[5]))
                          &(con_tr_feature['BSID7']==int(can_temp[6]))]
            
            match_ss_c= match_c[(match_c['rss_level_1']== ss_list[0]) & (match_c['rss_level_2']== ss_list[1]) 
                            & (match_c['rss_level_3']==ss_list[2])& (match_c['rss_level_4']==ss_list[3]) 
                            & (match_c['rss_level_5']==ss_list[4]) & (match_c['rss_level_6']==ss_list[5])
                            & (match_c['rss_level_7']==ss_list[6])]
            if match_ss_c.shape[0]>0:
                one_list.append(float(match_ss_c[match_ss_c['conf']==1].shape[0])/float(one_num))
            else:
                one_list.append(float(match_c[match_c['conf']==1].shape[0])/float(one_num))
        
        else:
            one_list.append(adaptive_emission_pro(can_j, match, bs_list, ss_list, can_bs_row, one_num, 1))
        
    else:
       
        match_ss= match[(match['rss_level_1']== int(ss_list[0])) & (match['rss_level_2']==int(ss_list[1])) & (match['rss_level_3']==ss_list[2])
             & (match['rss_level_4']==ss_list[3]) & (match['rss_level_5']==ss_list[4]) & (match['rss_level_6']==ss_list[5]) & (match['rss_level_7']==ss_list[6])]
        if match_ss.shape[0]>0:
            one_list.append(float(match_ss[match_ss['conf']==1].shape[0])/float(one_num))
        else:
            one_list.append(float(match[match['conf']==1].shape[0])/float(one_num))

In [42]:
one_list2 = np.array(one_list)
np.savetxt('one_list.txt', one_list2)
# one_list2 = np.loadtxt('one_list.txt')
# one_list2 = one_list2.tolist()

In [43]:
total_ob_te['conf_ad_em_pro_1'] = one_list2

In [44]:
con_te_feature['gid'] = con_te_j
con_te_feature['p_gid'] = pred_con_te
con_tr_feature['gid'] = con_tr_j
con_tr_feature['p_gid'] = pred_con_tr

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [45]:
trajs_tr = con_tr_feature.groupby(['TrajID'])

In [46]:
time_list = []
st_mat = np.zeros((14,2,2)) #0-0 0-1 1-0, 1-1
for trajid, traj in trajs_tr:
    traj = traj.sort_values(by=['Timestamp'],ascending=True)
    t_time = traj['Timestamp'].values
    conf = traj['conf'].values
    for i in range(traj.shape[0]-1):
        time_list.append(compute_time_interval(t_time[i], t_time[i + 1]))
        idx = int(compute_time_interval(t_time[i], t_time[i + 1])/5)
        if idx >12:
            idx = 13
        if conf[i]==0 and conf[i+1]==0:
            st_mat[idx, 0, 0] +=1
        if conf[i] ==0 and conf[i+1] ==1:
            st_mat[idx, 0, 1] +=1
        if conf[i]==1 and conf[i+1]==0:
            st_mat[idx, 1, 0] +=1
        if conf[i] ==1 and conf[i+1] ==1:
            st_mat[idx, 1, 1] +=1

1546925336
1546925358
1547024524
1547024567
1547024599
1547024968
1547025033
1547101168
1546925838
1552346614
1552346625
1552346657
1552346668
1554339545
1554339597
1554339646
1554339668
1554084857
1554084945
1554084994
1554085009
1554085020
1554085078
1554085100
1551658360
1551658371
1551658424
1556636841
1555319733
1555319743
1555319753
1555319753
1555319788
1555319798
1555319809
1555319820
1555319831
1555319853
1554371508
1554371532
1554371576
1554371587
1554371598
1554371693
1554371737
1554371748
1554371780
1554371813
1554371824
1554372273
1550654357
1546079349
1551828771
1551263168
1551263245
1551263277
1551263359
1551263457
1551263499
1551263521
1551263560
1546670230
1546670240
1546670251
1546670306
1548417444
1548417562
1548417639
1548247626
1548247651
1548247695
1548247727
1555296164
1555296173
1555296259
1555296302
1559886259
1559886279
1548569726
1546573261
1546573591
1552354978
1552355031
1552355199
1552355232
1552355243
1552355276
1552355319
1552355341
1550627296
1547118227

1548726356
1548726388
1551497952
1551497963
1551497995
1551498001
1551498025
1551498043
1551498043
1551498137
1551498148
1551498150
1551498174
1551498202
1551498485
1551498542
1551498542
1551498552
1551498561
1551498561
1551498583
1551498597
1551498608
1551498643
1551498672
1551498683
1551498750
1551498765
1551498786
1551498793
1551498793
1551498911
1551498956
1546667417
1557977228
1557977291
1557977301
1557977334
1551767549
1547028571
1547440463
1547438174
1557452869
1557453013
1557453073
1557453084
1557453095
1557453218
1546595342
1546595353
1546595566
1546595723
1546595745
1548239115
1548239147
1548239202
1548239265
1548239430
1548239452
1548239463
1548239485
1548239496
1557232611
1557232632
1557290943
1557291008
1557291787
1557291815
1557291877
1557291883
1557292027
1557292062
1557292077
1557292159
1557292166
1548668742
1548668753
1548668775
1560788440
1560788593
1552399622
1552399633
1552094611
1552094661
1552094704
1552094759
1555412246
1555412379
1555412400
1555412405
1555412489

1551783341
1546933990
1546934032
1546934043
1546934081
1546934156
1551484085
1551484085
1551484107
1551484152
1551484162
1551484211
1551484222
1551484222
1551484233
1551484568
1551484579
1551484579
1548208551
1547873497
1546910334
1546910377
1546910388
1546910399
1546910443
1546910491
1546910491
1546910502
1546910512
1546910533
1546910544
1546910544
1546910555
1546910588
1546910598
1547017423
1547017552
1547017552
1547017695
1547017847
1547017982
1547018015
1547018015
1547018037
1547018048
1547018048
1547709943
1559863833
1559863875
1553929408
1547301683
1553061407
1553061418
1553061449
1553062156
1551922701
1551922733
1551922776
1553051433
1551594382
1553681223
1553681260
1551088635
1547703226
1547703262
1547703434
1547703444
1548999310
1548999354
1548999695
1548999706
1547887829
1547887870
1547887881
1546744197
1555465060
1555465081
1555465117
1555465223
1555465289
1555465322
1555465615
1555474143
1549590755
1549590819
1549590858
1549590868
1549611055
1549611066
1549611197
1549611201

1557470456
1557470514
1557470536
1557470579
1557470624
1557470675
1557470696
1557470707
1557470717
1557470750
1549843893
1549843915
1547964400
1547964454
1547964475
1554948432
1547951913
1546422219
1546422251
1546422273
1546422294
1546422315
1546422349
1546422393
1548221956
1548222000
1548222177
1548228614
1557569686
1557569980
1557570023
1555061245
1555061533
1555061553
1555061618
1555061838
1555061849
1555061903
1555061936
1555062088
1555062109
1555062141
1555062205
1552352090
1552352358
1552352391
1552352422
1552352502
1547804579
1547804633
1547804655
1547804677
1547804710
1547804743
1547804807
1547804840
1547804862
1554954241
1554958666
1554958677
1554958730
1554958799
1554963534
1554963551
1554963770
1552382591
1552382623
1552382645
1552382710
1552382742
1555736919
1555737006
1555737039
1555737278
1555737310
1555737387
1555737397
1555737586
1547790285
1547790647
1547790658
1547790678
1557309817
1546164137
1546164175
1546164208
1546164241
1546164353
1549447420
1549447431
1549447431

1552553393
1550396963
1550396995
1550397006
1550397017
1550397028
1550397072
1550397281
1550397373
1550397384
1550397428
1550397439
1555577713
1555577745
1555577858
1555577869
1555577882
1548399913
1548399924
1554032021
1554032066
1554032131
1554032164
1554032175
1554032186
1554032245
1554032371
1554032436
1543478586
1543478625
1543478658
1543478871
1543478882
1552526206
1552526534
1552526598
1552526620
1552526627
1552526885
1552981021
1552981081
1552981092
1552981103
1552981230
1559712126
1559712148
1559712203
1559712225
1559712233
1559712244
1559712255
1557009193
1557009248
1556253793
1556253814
1556253847
1556253847
1556253869
1556253889
1556253921
1556253943
1556253976
1556253998
1556254020
1556254160
1546839331
1546397884
1546397945
1553466497
1553466519
1553466808
1553466851
1553466884
1553466895
1553466960
1553466982
1553466993
1553467025
1550117813
1550117878
1550117954
1555320531
1552032728
1552032893
1552033158
1552033191
1552033202
1552033213
1552033235
1552033289
1550555037

...

1554376545
1554376556
1554376566
1554376610
1554376621
1554376819
1554376839
1554376850
1554376969
1554377011
1547696530
1547696561
1547696572
1554718561
1554718572
1547464482
1547464534
1547464545
1550883018
1555640097
1555640118
1557877429
1557877451
1557877495
1551176035
1551176057
1551176068
1554023432
1554023519
1554023541
1554023585
1554023659
1554023689
1554023733
1554023806
1554023817
1554023827
1554023849
1554024040
1554024050
1554024133
1547776435
1547776620
1547776640
1547776650
1547776661
1555675859
1555675880
1555675903
1555675946
1555675990
1555676022
1555676054
1555676065
1555676086
1555676152
1548583075
1560821399
1560821432
1560829876
1560829887
1548396604
1548396718
1548396883
1548396958
1548396989
1548397000
1548397021
1548397032
1557802641
1557802720
1557803892
1557805723
1556604833
1556604865
1556604973
1556605028
1556605072
1556605318
1550321090
1550321241
1550321252
1555635127
1552922697
1552922734
1552922745
1551249009
1551249076
1551249108
1551249172
1551249204

1546576796
1546576814
1546576922
1560666692
1560666919
1554441935
1554441946
1554442267
1554442309
1554442320
1554442340
1554442454
1552004920
1552004940
1548201347
1548201357
1550977239
1550977261
1550977283
1550977283
1550977304
1550977304
1550977326
1550977326
1550977337
1550977337
1550977370
1550977381
1550977381
1560828782
1560828913
1560828945
1560829105
1559136426
1548328230
1548328252
1548328274
1554082229
1554182951
1554183560
1554183613
1554183731
1554183753
1554183828
1554183850
1554184129
1557383728
1557383746
1550541924
1550542120
1550542130
1550542180
1550542212
1550542247
1550542257
1550542300
1550542425
1553498134
1553498145
1553498202
1553498428
1550388126
1550388294
1550388323
1550388554
1548898555
1548898567
1548898620
1561288345
1561288378
1561288421
1561288443
1561288476
1561288591
1561288623
1561288645
1561288656
1551018317
1551018328
1551018352
1551018363
1551018385
1551018396
1551018396
1551018407
1551018443
1551018454
1553835742
1553835824
1551325293
1551325303

1555393776
1555393832
1555393859
1555393881
1555393902
1555393902
1555393913
1555393923
1555393923
1555393999
1555394416
1555394416
1555394464
1555394464
1546421714
1546421907
1546421929
1546421985
1546421996
1546422008
1546422018
1555746849
1550744697
1550744708
1550744710
1550744853
1550744961
1550745058
1550745159
1550745197
1550745218
1550745228
1555395450
1555395458
1555395472
1555395538
1555395586
1555395729
1558047110
1558047151
1558047170
1558065986
1558066017
1558066028
1558066229
1558066301
1558066312
1558066333
1558066431
1558066453
1558066484
1558066537
1558066558
1558066619
1558066647
1557567368
1557567499
1557567521
1557567532
1557567643
1557567654
1554702312
1546839211
1546839222
1546839244
1546839266
1546839277
1546839309
1546839320
1550302287
1550302319
1550302448
1550302480
1550302598
1550302759
1550302769
1552615313
1552615346
1552615348
1551603323
1547167849
1547167899
1547167974
1547167995
1547168118
1547168314
1547168335
1547168356
1547168378
1547168419
1547168452

1557625623
1557625634
1557625678
1557625711
1557484563
1557491895
1557491969
1557492055
1557492087
1557492098
1557492168
1557492657
1557492721
1557492754
1557492776
1557492873
1557493057
1557493153
1557493207
1557493412
1557493465
1557493476
1557493530
1557493541
1547134909
1547134920
1546600202
1546600255
1546600277
1552630949
1548317911
1548317956
1548839356
1548839427
1548839735
1548839795
1548839853
1554954962
1554954982
1554955068
1554955079
1554955111
1554955255
1552639183
1552639195
1552639206
1552639293
1552639376
1552639747
1552639785
1550926854
1550927008
1550927222
1550927233
1557325705
1555545581
1554739252
1554739317
1557970584
1557970755
1557970766
1557970917
1550470142
1550470255
1550470291
1550470334
1550470355
1550470411
1550470443
1554937507
1554937550
1554937619
1554937629
1554937662
1553476002
1553476035
1551913452
1551913463
1551913485
1551913781
1552022035
1552022079
1552022110
1552022132
1555834159
1555834181
1555834203
1555834302
1555834399
1546815651
1546815672

1554860188
1554860252
1554860262
1554860299
1554860342
1554860504
1554860652
1554860714
1559974992
1559975003
1559975250
1559975283
1554867450
1554867602
1550561724
1550561802
1550561899
1550913035
1550913068
1550913081
1550990343
1550990354
1550990372
1550990419
1550990430
1550990440
1550990451
1550990473
1550990484
1550990534
1550990544
1552879755
1552879830
1552879841
1552879852
1552879863
1552879918
1552879940
1552880040
1552880051
1552880062
1552493808
1551142171
1551142403
1547122265
1547122276
1547122456
1547122467
1547208717
1547208739
1547208792
1552297668
1552298006
1550556704
1550556728
1547775064
1547775074
1547775085
1547775153
1547775164
1547775197
1551352154
1551352165
1551352175
1551352175
1551352186
1551352186
1551352229
1551352262
1551352327
1551352338
1551352422
1558836308
1558836339
1558836769
1558836813
1552448437
1552448485
1558642624
1558642646
1548418500
1548418533
1548418593
1548418629
1548418705
1559704775
1557457209
1557457231
1547782629
1547782650
1547782861

In [47]:
trajs_te = con_te_feature.groupby(['TrajID'])

In [48]:
pred_list_t_idx = []
#st_mat = np.zeros((14,2,2)) #0-0 0-1 1-0, 1-1
pred_list = []
p_g_list = []
t_g_list = []
for trajid, traj in trajs_te:
    traj = traj.sort_values(by=['Timestamp'],ascending=True)
    t_time = traj['Timestamp'].values
    conf = traj['conf'].values
    idx_list = []
    for i in range(traj.shape[0]-1):
        time_list.append(compute_time_interval(t_time[i], t_time[i + 1]))
        idx = int(compute_time_interval(t_time[i], t_time[i + 1])/5)
        if idx >12:
            idx = 13
        idx_list.append(idx)
    pred_list.append(traj[['BSID','rss_level_1','BSID2','rss_level_2',
               'BSID3','rss_level_3','BSID4','rss_level_4',
               'BSID5','rss_level_5','BSID6','rss_level_6',]].values)
        
    pred_list_t_idx.append(idx_list)
    p_g_list.append(traj['p_gid'].values)
    t_g_list.append(traj[['Longitude','Latitude']].values)

In [49]:
init_prob = [float(con_tr_feature[con_tr_feature['conf']==0].shape[0])/con_tr_feature.shape[0],
             float(con_tr_feature[con_tr_feature['conf']==1].shape[0])/con_tr_feature.shape[0]]

In [50]:
def hmm_viterbi(st_mat, init_prob, emit_prob, obs_seq, t_list):
    Nstate = 2
    Nobs = int(emit_prob.shape[0])
    T = len(obs_seq)
    
    partial_prob = np.zeros((Nstate,T))

    path = np.zeros((Nstate,T))

    for i in range(Nstate):
        partial_prob[i,0] = init_prob[i] * emit_prob[obs_seq[0], i]
        path[i,0] = i


    for t in range(1,T,1):
        newpath = np.zeros((Nstate,T))
        for i in range(Nstate):
            prob = -1.0
            for j in range(Nstate):
                nprob = partial_prob[j,t-1] * st_mat[t_list[t-1], j, i] * emit_prob[obs_seq[t], i]
                if nprob > prob:
                    prob = nprob
                    partial_prob[i,t] = nprob
                    newpath[i,0:t] = path[j,0:t]
                    newpath[i,t] = i
        path = newpath
    
    prob = -1.0
    j = 0
    for i in range(Nstate):
        if(partial_prob[i,T-1] > prob):
            prob = partial_prob[i,T-1]
            j = i

    return path[j,:]

In [51]:
em_pro = total_ob_te
em_pro = em_pro.reset_index(drop = True)

In [52]:
emit_prob = em_pro[['conf_ad_em_pro_0', 'conf_ad_em_pro_1']].values

In [53]:
num_g = rg.n_grid
w = np.zeros((num_g, num_g))

test = zip(pred_con_tr, con_tr_j, error_tr)
tj_r = {}
for ss in test:
    #print ss
    #w[ss[0]][num_g]=w[ss[0]][num_g]+1
    #w[ss[0]][ss[1]]=w[ss[0]][ss[1]]+1
    if ss[0]!=ss[1] and ss[2]>50:
         w[ss[0]][ss[1]] +=1
        #w[ss[0], ss[1]] += 1
#     if not tj_r.has_key(ss[0]):
    if ss[0] not in tj_r:
        tj_r[ss[0]]=set()
    tj_r[ss[0]].add((ss[1]))

standard_point=[]


for item in tj_r:
    if (len(tj_r[item])==1) and (item in tj_r[item]):
        standard_point.append(item)

for idx in range(num_g):
    if idx not in tj_r:
        tj_r[idx]=set()
        tj_r[idx].add((idx))

g_list=rg.gridlist

In [54]:
g_obs = {}
for idx, row in con_tr_feature.iterrows():
    obs = row[['BSID','rss_level_1','BSID2','rss_level_2',
               'BSID3','rss_level_3','BSID4','rss_level_4',
               'BSID5','rss_level_5','BSID6','rss_level_6',]].values
    gid = int(row['gid'])
    match = em_pro[(em_pro['BSID']==obs[0]) & (em_pro['rss_level_1']==obs[1]) & 
                      (em_pro['BSID2']==obs[2]) & (em_pro['rss_level_2']==obs[3]) & 
                      (em_pro['BSID3']==obs[4]) & (em_pro['rss_level_3']==obs[5]) & 
                      (em_pro['BSID4']==obs[6]) & (em_pro['rss_level_4']==obs[7]) & 
                      (em_pro['BSID5']==obs[8]) & (em_pro['rss_level_5']==obs[9]) & 
                      (em_pro['BSID6']==obs[10]) & (em_pro['rss_level_6']==obs[11])]
    if match.shape[0]>0:
        obs_idx = (gid, int(match.index[0]))
        if obs_idx not in g_obs:
            g_obs[obs_idx]=1
        else:
            g_obs[obs_idx] +=1

In [55]:
def repair(te_pred, cof_list, o_list):
    i=0
    te_pred_n=[]
    while i < len(cof_list):
        if cof_list[i]==0:
            pred_temp = te_pred[i]
            if pred_temp in standard_point:
                te_pred_n.append(te_pred[i])
            else:
                repair_r = list(w[te_pred[i], :])
                max_idx = repair_r.index(np.max(repair_r))
                if (max_idx, o_list[i]) in g_obs:
                    te_pred_n.append(int(max_idx))
                    #print 'ok'
                else:
                    
                    te_pred_n.append(te_pred[i])
                
        else:
            te_pred_n.append(te_pred[i])
        i+=1
            
    #print len(te_pred_n)
    return te_pred_n

In [56]:
import operator
error_list = []
error_new =[]
i=0
for row, time, raw, gt in zip (pred_list, pred_list_t_idx, p_g_list, t_g_list):

    o_seq = []
    for obs in row:
        #print obs
        match = em_pro[(em_pro['BSID']==obs[0]) & (em_pro['rss_level_1']==obs[1]) & 
                      (em_pro['BSID2']==obs[2]) & (em_pro['rss_level_2']==obs[3]) & 
                      (em_pro['BSID3']==obs[4]) & (em_pro['rss_level_3']==obs[5]) & 
                      (em_pro['BSID4']==obs[6]) & (em_pro['rss_level_4']==obs[7]) & 
                      (em_pro['BSID5']==obs[8]) & (em_pro['rss_level_5']==obs[9]) & 
                      (em_pro['BSID6']==obs[10]) & (em_pro['rss_level_6']==obs[11])]
        o_seq.append(match.index[0])
    pred = raw
    true = gt
    cof_list = hmm_viterbi(st_mat, init_prob, emit_prob, o_seq, time)
    r_pred = repair(pred, cof_list, o_seq)
    
   
    te_predp = np.array([rg.grid_center[idx] for idx in r_pred])
    #tps = np.array([rg.grid_center[idx] for idx in true])
   
    error_tep = [distance(pt1, pt2) for pt1, pt2 in zip(te_predp, true)]
    #print np.mean(error_trp), np.mean(error_tep)
   
    for t in error_tep:
        #print t
        error_new.append(t)
    i+=1
    #print pred
    #print 'a', r_pred


In [57]:
err= sorted(error_new)

In [58]:
print(np.median(error_new), np.mean(error_new), err[int(len(err) * 0.9)])

(50.700000000000003, 102.81418721073013, 239.3)
